## 推論エンジンのpythonへの移植
Steven John Metsker著「<a href="https://www.amazon.com/Building-Parsers-Java%C2%BF-Steven-Metsker/dp/0201719622">JAVAによるサーバ構築技法（Building Parsers With Java）</a>」に付属している推論エンジンとパーサをpythonに移植しました。
同ソースコードは、以下のURLで公開されていました（mavenでコンパイルできるようにしたもので著者以外の人による版と思われます）。
- https://github.com/sujitpal/bpwj

CDROMのreadme.txtによるとコードは自由に使ってよいと言うことでこれをベースにpythonにエンジンを移植していきます。
```
Copyright
---------
The code on the CD is free. It is copyrighted, so you may not claim that 
you wrote the code. Otherwise you may use the code as you wish.
```

エンジンのポーティングは、「JAVAによるサーバ構築技法」の12章をベースに行いました。
この章の例題を移植したソースに適応して、動作を確認しながらポーティング作業を進めました。

### 移植のポイント
エンジンのメイン機能は、単一化（Unification）とバックトラック処理にあります。

論理エンジンは、Structureクラスをベースとして作られています。
構造（Structure）は、関数子（functor）と項（terms）の配列で構成されています。

項を持たず、関数子だけをもつ構造をアトム（Atom）と呼びます。

エンジンの最終的なポート結果は、engine.pyにあります。

## Unificationの移植
Javaのソースは、クラス毎に定義され、相互参照をしているため、Pythonで一つのファイルで定義するには、そのクラスがグローバルに定義されているものとしてコーディングする必要があります。

最初に必要なライブラリをインポートします。

```python
import copy
from collections import defaultdict
import time
```

次にUnificationを移植します。bpwj/src/main/java/sjm/engine/にエンジン周りのJavaクラスが定義されています。

Unification.javaをPythonに移植します。
Javaソースのvectorをself._variablesとして実装しました。

Unificationクラスは、変数（Variable）の単一化の結果を_variablesに保持します。


Unificationクラスの移植では、クラス変数emptyの初期化をクラスインスタンス生成メソッド__new__の中で実装することです。

```python
class Unification():
    empty = None
    def __new__(cls, *args, **kargs):

        if cls.empty == None:
            cls.empty = super(Unification, cls).__new__(cls)
            cls.empty._variables = []
        return super(Unification, cls).__new__(cls)

    def __init__(self, v = None):
        self._variables = []
        if v != None:
            self.addVariable(v)
```


## Structureの移植
Sturctureクラスのコンストラクターは、以下のようになります。

```python
class Structure:
    # コンストラクター
    def __init__(self, functor, terms = []):
        self.functor = functor
        self.terms = terms
        if terms != None and len(terms) > 0:
            self.terms = terms
```

これをjavaの定義と比較すると、javaでは複数のコンストラクターが定義できるのに対して、pythonでは__init__の１つしか定義できません。この点がpythoへの移植の厄介なところでした。
```java
public class Structure implements Term {
	protected Object functor;
	protected Term[] terms;

    public Structure(Object functor) {
        this(functor, new Term[0]);
    }
    public Structure(Object functor, Term[] terms) {
        this.functor = functor;
        this.terms = terms;
    }    
}
```


engine.pyからStructureクラスを抜粋します（コメントを除く）。

ここで注目して欲しいのは、以下の３つです。
- コンストラクター（__init__）
- 単一化（unify）
- 同一判定（__eq__）

それと__str__は、インスタンスの表示の時に役に立つので、最初に実装するのがポイントです。また、一度にすべてのメソッドを実装するのではなく、最低限のメソッドから実装し、動作を確認しながら薦めると良いでしょう。

```python
class Structure:
    # 以下抜粋
    # コンストラクター
    def __init__(self, functor, terms = []):      
        self.functor = functor
        self.terms = terms
        if terms != None and len(terms) > 0:
            self.terms = terms
    # 単一化
    def unify(self, s):
        if isinstance(s, Structure):
            if not self.functorAndArityEquals(s):
                return None
            u = Unification()
            others = s.terms
            for i in range(len(self.terms)):
                subUnification = self.terms[i].unify(others[i])            
                if subUnification == None:
                    u.unbind()
                    return None
                u.append(subUnification)
            return u
        elif isinstance(s, Variable):
            v = s
            return v.unify(self)
        else: # Term
            t = s
            return t.unify(self)
    # 同一判定
    def __eq__(self, o):       
        if type(self) != type(o):
            return False
        s = o
        if not self.functorAndArityEquals(s):
            return False
        for i in range(self.arity()):
            if not self.terms[i].__eq__(s.terms[i]):
                return False
        return True
```

### 動作確認
「JAVAによるサーバ構築技法」のすぐれたところは、クラスの定義と動作確認が実装されていることです。
- src/main/java/sjm/examples/に動作確認用のクラスが定義されています。

ShowStructure.javaをPythonに移植して動作を確認します。

Structureを使ったAtomの例として、denver（デンバー）を定義すると以下のようになります。

In [1]:
from engine import *

denver = Structure("denver")
print(denver)

denver


構造を使ってcityの地名（Name）と標高（altitude）を定義すると、以下のようになります。

In [2]:
denver = Structure("denver")
alititude = Structure(5280)
city = Structure("city", [denver, alititude])
print(city)

city(denver, 5280)


## 変数(Variable)の移植

次に変数（Variable）を移植します。

ここでのポイントは以下のメソッドです。
- コンストラクター（__init__）
- 単一化（unify）
- 単一化の解除（unbind）



```python
class Variable: 
    # 抜粋
    def __init__(self, name):    
        self.name = name
        self.instantiation = None
        self.id = f"{name}_{str(time.time())}"

    def unify(self, s):     
        structureCls = globals()['Structure']
        if isinstance(s, Variable):
            v = s
            if self is v:
                return Unification()
            elif self.instantiation != None:
                return self.instantiation.unify(v)
            elif v.instantiation != None:
                return v.instantiation.unify(self)
            self.instantiation = v
            return Unification(self)
        elif isinstance(s, structureCls):
            if self.instantiation != None:
                return self.instantiation.unify(s)
            self.instantiation = s
            return Unification(self)
        else: # Term
            t = s
            return t.unify(self)
        
    def __eq__(self, o):      
        if not isinstance(o, Variable):
            return False
        v = o
        if self.name != v.name:
            return False
        if self.instantiation == None:
            return v.instantiation == None
        return self.instantiation.__eq__(v.instantiation)
```

### 単一化（Unification）
変数（Variable）の単一化の結果を保持するクラスがUnificationです。

JAVAによるサーバ構築技法から単一化の方法をまとめると以下の通りです。
- ２つの構造が単一化できるためには、両者が同じ関数子を持ち、項の数が等しく、
対応する項同士が単一かできる必要がある
- インスタンス化されていない変数を特定の構造に単一化すると、その構造が変数の値になる
- インスタンス化された変数は、インスタンス値に単一化を依頼することで、自身の単一化を実行する

先に作成した構造体cityと変数name, altの単一化をすると、各変数のinstance属性に構造体denverとalititudeがセットされていることを確かめてみましょう(ShowStructureUnification.javaの移植)。

In [3]:
from engine import *

# Variableの確認
name = Variable("Name")
alt = Variable("Altitude")
vCity = Structure("city", [name, alt])
print(vCity)

city(Name, Altitude)


vCityに先に定義したcityをunifyメソッドを実行すると、vCityの構造体（Structure）の変数name, altにcityの値（denver, 5280）がそれぞれ単一化されます。

In [4]:
vCity.unify(city)
print(f"name={str(name)}, alt={str(alt)}")

name=denver, alt=5280


In [5]:
# 以下のコマンドをデバッガで実行すると、変数name, altに
# セットされているinstatiationに構造cityのdenver, alititude
# であることが確認できます。
print(name.instantiation)
print(alt.instantiation)

denver
5280


デバッガ画面で、name.instantiationに構造(0x110e51930)がセットされ、それが構造(denver)であり、そのfunctorの値がdenverであることが確認できます。

<img src="images/unification.png">

## Factの移植
事実（Fact）は、変数を含まない構造（Struct）と説明されています。

Factには、様々なコンストラクターが用意されており、事実の記述が簡単にできるように配慮されています。

Javaのコンストラクターには、以下のメソッドが提供されています。

- Fact(functor: Object)
- Fact(functor: Object, term0: Object)
- Fact(functor: Object, term0: Object, term1: Object)
- Fact(functor: Object, terms[]: Object)
- Fact(functor: Object, terms[]: Fact)

これをpythonのコンストラクター（__init__）で以下のように定義しています。

```python
class Fact(Structure):    
    def __init__(self, functor, *args):     
        if len(args) == 0:
            super().__init__(functor)
        elif len(args) == 1:            
            if isinstance(args[0], list):
                first = args[0]
                if len(first) > 0 and isinstance(first[0], Fact):
                    super().__init__(functor, first)
                else:
                    atoms = [Atom(o) for o in first]
                    super().__init__(functor, atoms)
            elif isinstance(args[0], Fact):
                super().__init__(functor, [args[0]])
            else:
                super().__init__(functor, [Atom(args[0])])
        elif len(args) == 2:
            o1 = args[0]
            o2 = args[1]
            if isinstance(o1, Atom):
                super().__init__(functor, [o1, o2])
            else:
                super().__init__(functor, [Atom(o1), Atom(o2)])

```

### Factの動作確認
Factの動作確認用にShowFactで定義されている処理をPythonで実行してみます。

多様なコンストラクターの例となっています。

In [6]:
from engine import *

d = Fact("city", [Fact("denver"), Fact(5280)])
j = Fact("city", "jacksonville", 8)

print(d)
print(j)

city(denver, 5280)
city(jacksonville, 8)


## ProgramとQueryの移植
Prologでは、プログラム（Program）は公理が集まったものであり、公理は事実（Fact）と規則（Rule）のいずれかとされています。

問い合わせ（Query）は、プログラムの公理を使って規則を証明するものです。

ShowProgram.javaをPythonに移植して、動作をみてみましょう。

### 問い合わせの実行
問い合わせ（Query）に次の証明するものがあるかを検査するメソッドがcanFindNextProofです。

In [7]:
facts = [
    Fact("city",    "abilene",      1718),
    Fact("city",    "addis ababa",  8000),
    Fact("city",    "denver",       5280),
    Fact("city",    "flagstaff",    6970),
    Fact("city",    "jacksonville", 8),
    Fact("city",    "leadville",    10200),
    Fact("city",    "madrid",       1305),
    Fact("city",    "richmond",     19),
    Fact("city",    "spokane",      1909),
    Fact("city",    "wichita",      1305),
]

p = Program(facts)
name = Variable("Name")
height = Variable("Height")
s = Structure("city", [name, height])

try:
    q = Query(p, s)
    while q.canFindNextProof():
        print(f"{name} is about {height} feet above sea level.")
except Exception as e:
    print(e)



abilene is about 1718 feet above sea level.
"addis ababa" is about 8000 feet above sea level.
denver is about 5280 feet above sea level.
flagstaff is about 6970 feet above sea level.
jacksonville is about 8 feet above sea level.
leadville is about 10200 feet above sea level.
madrid is about 1305 feet above sea level.
richmond is about 19 feet above sea level.
spokane is about 1909 feet above sea level.
wichita is about 1305 feet above sea level.


## 証明
問い合わせ（Query）による証明では、スコープ（Scope）に証明で使用する規則（Rule）で使われる変数をコピーし、管理しています。

canFindNextProofでは、事実（Fact）に規則（Rule）を単一化し、変数に事実の情報をセットしていきます。この処理をすべての事実に繰り返し、失敗したら、ルールが成り立っている箇所まで戻り（バックトラック）ってルールが証明を繰り返します。

exampleのShowProof.javaをPythonに移植して動作を確認します。

In [8]:
facts = [
    Fact("charge", "athens", 23),
    Fact("charge", "sparta", 13),
    Fact("charge", "milos", 17),
]
charges = Program(facts)

facts = [
    Fact("customer", "Marathon Marble", "sparta"),
    Fact("customer", "Acropolis Construction", "athens"),
    Fact("customer", "Agora Imports", "sparta"),
]
customers = Program(facts)

p = Program()
p.append(charges)
p.append(customers)

city = Variable("City")
fee = Variable("Fee")
name = Variable("Name")
s1 = Structure("charge", [city, fee])
s2 = Structure("customer", [name, city])
q = Query(p, [s1, s2])

プログラム（Program）pと問い合わせ（Query）をプリントし、事実と問い合わせの規則をPrologとして表示します。 

In [9]:
print(p)

charge(athens, 23);
charge(sparta, 13);
charge(milos, 17);
customer("Marathon Marble", sparta);
customer("Acropolis Construction", athens);
customer("Agora Imports", sparta);


In [10]:
print(q)

charge(City, Fee), customer(Name, City)


### 例題の実行
この例題は、顧客（customer）の名前nameと住所（地名city）の一覧、運賃表（Charge）の地名cityと運賃Feeの一覧から、送り先（地名City）と運賃Fee、顧客の名前Nameを出力します。

最初にcustomerの一覧から
- customer("Acropolis Construction", athens)に変数Name, Cityが単一化し、値がセットされます。

次にchargeの一覧から地名city=athensと単一化する以下の事実から変数Feeに23がセットされます。
- Charge(athens, 23)

これをすべての事実を照会しながら、該当する結果を出力すると以下のようになります。

In [11]:
try: 
    while q.canFindNextProof():
        print(f"City: {city}")
        print(f"Fee: {fee}")
        print(f"Name: {name}") 
except Exception as e:
    print(e)

City: athens
Fee: 23
Name: "Acropolis Construction"
City: sparta
Fee: 13
Name: "Marathon Marble"
City: sparta
Fee: 13
Name: "Agora Imports"


## 関数の追加
Prologエンジンとして使用するために、いくつかの組み込み関数を追加しています。

- 比較（>, <, = >=, <=, !=）
- 算術（+, -, *, /, %）

関数の場合、Getewayのサブクラスとして実装します。

最初に比較の動作確認をShowComarison.javaを移植して、動作を確認します。

In [12]:
alt1 = Atom(5280)
alt2 = Atom(19)

q1 = Query(None, Comparison(">", alt1, alt2))
print(f"q1: {q1.canFindNextProof()}")

q2 = Query(None, Comparison(">", Atom("denver"), Atom("richmond")))
print(f"q2: {q2.canFindNextProof()}")

q1: True
q2: False


算術関数は、比較で使われたComparisionTermのサブクラスArithmeticTermインタフェースを実装していますが、Pythonで直に必要なメソッドを追加しています。

算術関数の動作確認は、ShowArithmetic.javaをPythonに移植して行います。

In [13]:
a = Fact(1000)
b = Fact(999)
x = ArithmeticOperator("*", a, b)
y = ArithmeticOperator("+", x, b)

print(y)
print(y.eval())

+(*(1000, 999), 999)
999999


順番は前後しますが、評価（eval）は、第1引き数の変数に第2項の値を単一化する関数です。

以下の例では、Evaluation(baby, diff)でdiffの結果を評価し変数babyに単一化しています。
式を評価するには、式の証明（canFindNextProof）を依頼します。

In [14]:
you = Variable("You")
youAndBaby = Variable("YouAndBaby")
baby = Variable("Baby")

diff = ArithmeticOperator("-", youAndBaby, you)

e = Evaluation(baby, diff)
print(e)

you.unify(Fact(185))
youAndBaby.unify(Fact(199))
print(e)

e.canFindNextProof()
print(f"That baby weight about {baby} pounds.")

#(Baby, -(YouAndBaby, You))
#(Baby, -(199, 185))
That baby weight about 14 pounds.


証明では、証明の結果を否定するNot機能があると便利です。

ただし、Notの実装は少し複雑でNotを評価するためにConsultingNotクラスを作成し、Notの評価時にConsultingNotでラップするように実装しています。


動作確認は、独身の男性を求めるプログラムです。

In [15]:
p = Program()
x = Variable("X")
s0 = Structure("bachelor", [x])
s1 = Structure("male", [x])
s2 = Not("married", [x])
r0 = Rule([s0, s1, s2])
p.addAxiom(r0)

p.addAxiom(Fact("married", "jim"))

p.addAxiom(Fact("male", "jeremy"))
p.addAxiom(Fact("male", "jim"))

print(p)

b = Variable("B")
q = Query(p, Structure("bachelor", [b]))

while q.canFindNextProof():
    print(f"{b} is a bachelor")

bachelor(X) :- male(X), not married(X);
married(jim);
male(jeremy);
male(jim);
jeremy is a bachelor


無名変数は、任意の項と単一化されます：項の値を問わず真となります。Prologではアンダスコア（_）で表します。

In [16]:
m1 = Fact("marriage", [
    1,
    "balthasar",
    "grimedlda",
    14560512,
    14880711
])
m257 = Fact("marriage", [
    257,
    "kevin",
    "karla",
    19790623,
    "present"
])

p = Program()
p.addAxiom(m1)
p.addAxiom(m257)

id = Variable("Id")
hub = Variable("Hub")
a = Anonymous()

q = Query(p, Structure("marriage", [id, hub, a, a, a]))
while q.canFindNextProof():
    print(f"Id: {id}, Husband: {hub}")

Id: 1, Husband: balthasar
Id: 257, Husband: kevin


### Write関数
Gatewayのサブクラスとして、引き数の値を標準出力書き出す関数Writeを追加しました。

In [17]:
v = Variable("V")
v.unify(Fact(2))
e = Write("和=", ArithmeticOperator("+", Fact(1), v))
e.canFindNextProof()

和=3


True

### リスト
PrologのリストLispの（Car, Cdr)の要なツリー構造で実装されています。


In [18]:
snakes = Structure.list(["cobra", "garter", "python"])
print(f"A list of three snakes: {snakes}")

a = Variable("A")
b = Variable("B")
c = Variable("C")
abc = Structure.list([a, b, c])
print(f"\n... unifies whth: {abc}")

abc.unify(snakes)
print("\n... giving: \n" +
      f"A: {a}\n" +
      f"B: {b}\n" +
      f"C: {c}\n")

head = Variable("Head")
tail = Variable("Tail")
ht = Structure.list([head], tail)
print(f"... and unifies with: {ht}")

ht.unify(snakes)
print(f"... giving: \n" + 
      f"Head: {head} \n" +
      f"Tail: {tail}\n")

A list of three snakes: [cobra, garter, python]

... unifies whth: [A, B, C]

... giving: 
A: cobra
B: garter
C: python

... and unifies with: [Head|Tail]
... giving: 
Head: cobra 
Tail: [garter, python]



## 伝染病
この例題は、推論エンジンの特徴をうまく使った例です。

- 感染した人物をキスを伝染が発症します。
- キスは双方向に成立するため、新たにkissedという事実を使っています。

ルール内の変数定義は、各Structureの中で毎回定義します。Ruleのスコープ内で変数名の重複チェックが行われるため、Rule内では一意の変数を参照します。

In [19]:
from engine import *

facts = [
    Fact("kiss", ["jill", "james", "0321"]),
    Fact("kiss", ["julian", "jill", "0405"]),
    Fact("kiss", ["jill", "jasper", "0412"]),
    Fact("kiss", ["james", "jenni", "0420"]),
    Fact("kiss", ["julian", "judy", "0508"]),
    Fact("kiss", ["jed", "judy", "0614"]),
]

p = Program(facts)

infected = Fact("infected", ["julian", "0307"])
p.addAxiom(infected)

p.addAxiom(Rule([
    Structure("kissed", [Variable("Student1"), Variable("Student2"), Variable("Date")]), # :-
        Structure("kiss", [Variable("Student1"), Variable("Student2"), Variable("Date")])
]))

p.addAxiom(Rule([
    Structure("kissed", [Variable("Student1"), Variable("Student2"), Variable("Date")]), # :-
        Structure("kiss", [Variable("Student2"), Variable("Student1"), Variable("Date")])
]))

p.addAxiom(Rule([
    Structure("exposed", [Variable("Student"), Variable("Date")]), # :-
        Structure("infected", [Variable("Student"), Variable("Date1")]),
        Comparison(">", Variable("Date"), Variable("Date1"))
]))

p.addAxiom(Rule([
    Structure("exposed", [Variable("Student2"), Variable("Date2")]), # :-
        Structure("kissed", [Variable("Student2"), Variable("Student1"), Variable("Date1")]),
        Comparison(">", Variable("Date2"), Variable("Date1")),
        Structure("exposed", [Variable("Student1"), Variable("Date1")])
]))

# 定理の出力
print("定理:")
print(p)

# この変数名は、上記変数と被らないようにする必要がある
x = Variable("X")

print("出力結果:")
q = Query(p, Structure("exposed", [x, Atom("0615")]))
while q.canFindNextProof():
    print(f"Student = {x}")

定理:
kiss(jill, james, 0321);
kiss(julian, jill, 0405);
kiss(jill, jasper, 0412);
kiss(james, jenni, 0420);
kiss(julian, judy, 0508);
kiss(jed, judy, 0614);
infected(julian, 0307);
kissed(Student1, Student2, Date) :- kiss(Student1, Student2, Date);
kissed(Student1, Student2, Date) :- kiss(Student2, Student1, Date);
exposed(Student, Date) :- infected(Student, Date1), >(Date, Date1);
exposed(Student2, Date2) :- kissed(Student2, Student1, Date1), >(Date2, Date1), exposed(Student1, Date1);
出力結果:
Student = julian
Student = jed
Student = jill
Student = jasper
Student = judy


## ループ処理
次にループ処理の例題です。
Writeを追加し、途中の値を出力できるようにしました。

In [20]:
from engine import *

p = Program()
p.addAxiom(Rule([
    Structure("for", [Variable("X"), Variable("X"), Variable("Upper")]), # :-
        # 以下の比較で変数Xが単一化される
        Comparison("<=", Variable("X"), Variable("Upper")),
        #Write(Variable("X"))
]))
p.addAxiom(Rule([
    Structure("for", [Variable("X"), Variable("Lower"), Variable("Upper")]), # :-
        Comparison("<", Variable("Lower"), Variable("Upper")),        
        Evaluation(Variable("LowerPlusOne"), ArithmeticOperator("+", Variable("Lower"), Atom(1))),   
        Structure("for", [Variable("X"), Variable("LowerPlusOne"), Variable("Upper")])
]))

# 定理の出力
print("定理:")
print(p)

i = Variable("i")

print("出力結果:")
q = Query(p, Structure("for", [i, Atom(1), Atom(20)]))
while q.canFindNextProof():
    print(q.variables())

定理:
for(X, X, Upper) :- <=(X, Upper);
for(X, Lower, Upper) :- <(Lower, Upper), #(LowerPlusOne, +(Lower, 1)), for(X, LowerPlusOne, Upper);
出力結果:
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20


## ハノイの塔の例題
リングを移動する途中の様子を表すと以下のようになります。
1. n-1枚をAからBに移す
1. n枚目をAからCに移す
1. n-1枚をBからCに移す

<img src="images/hanoi.png">

これを推論エンジンで表現すると以下の様になります。

In [21]:
from engine import *

p = Program()
p.addAxiom(Rule([
    Structure("hanoi", [Atom(1), Variable("From"), Variable("To"), Variable("Via")]), # :-        
        Structure("move", [Atom(1), Variable("From"), Variable("To")])
]))
p.addAxiom(Rule([
    Structure("hanoi", [Variable("N"), Variable("From"), Variable("To"), Variable("Via")]), # :-      
        Comparison(">", Variable("N"), Atom(1)), 
        Evaluation(Variable("N1"), ArithmeticOperator("-", Variable("N"), Atom(1))),   
        Structure("hanoi", [Variable("N1"), Variable("From"), Variable("Via"), Variable("To")]),
        Structure("move", [Variable("N"), Variable("From"), Variable("To")]),
        Structure("hanoi", [Variable("N1"), Variable("Via"), Variable("To"), Variable("From")])
]))
p.addAxiom(Rule([
    Structure("move", [Variable("N"), Variable("From"), Variable("To")]),   
        Write("Move", Variable("N"), ": ", "from ", Variable("From"), " to ", Variable("To"))
]))

# 定理の出力
print("定理:")
print(p)

print("出力結果:")
q = Query(p, Structure("hanoi", [Atom(3), Atom("A"), Atom("B"), Atom("C")]))
while q.canFindNextProof():
    print(q.variables())

定理:
hanoi(1, From, To, Via) :- move(1, From, To);
hanoi(N, From, To, Via) :- >(N, 1), #(N1, -(N, 1)), hanoi(N1, From, Via, To), move(N, From, To), hanoi(N1, Via, To, From);
move(N, From, To) :- write(Move, N, ": ", "from ", From, " to ", To);
出力結果:
Move1: from A to B
Move2: from A to C
Move1: from B to C
Move3: from A to B
Move1: from C to A
Move2: from C to B
Move1: from A to B

